In [1]:
import torch

dataset_path = "/home/rafael/Área de trabalho/Linux/graph_dataset.pt"
dataset = torch.load(dataset_path)

print(f"Graphs loaded: {len(dataset)}")

/tmp/ipykernel_9446/3346811239.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load(dataset_path)


Graphs loaded: 620


In [ ]:
import sys
sys.path.append("/media/rafael/HD/orguel_ml_library")
from orguel_ml import BalanceClassWeights
from orguel_ml.ocr import OCRNetwork, character_set
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split

# Setup
epochs = 100
batch_size = 1
learning_rate = 0.005
weight_decay = 1e-4
smoothing_factor = 0.2
label_smoothing = 0.1
test_size = 0.1

# split the dataset:
train_data, validation_data = train_test_split(dataset, test_size=test_size, shuffle=True, random_state=42)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Character and font class weights
n_characters = len(character_set["characters"]["encoding"])
n_fonts = len(character_set["font"]["encoding"])
character_weights = BalanceClassWeights(train_data, device, smoothing_factor, classification_labels=n_characters)
font_weights = BalanceClassWeights(train_data, device, smoothing_factor, classification_labels=n_fonts)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batch_size)

model = OCRNetwork(cluster_aware=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.01, step_size_up=5, mode="triangular")
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, min_lr=1e-5, threshold=1e-5)

In [ ]:
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

# TensorBoard writer
writer = SummaryWriter(log_dir="TensorBoard")

# Training loop with Cross Entropy clearly shown
for epoch in range(epochs):
    model.train()
    acumulatedTotalLoss = 0
    acumulatedCharacterLoss, acumulatedFontLoss = 0, 0
    acumulatedHeightLoss, acumulatedRotationLoss, acumulatedInsertionLoss = 0, 0, 0
    acumulatedMAEHeight, acumulatedMAERotation, acumulatedMAEInsertion = 0, 0, 0
    acumulatedRMSEHeight, acumulatedRMSERotation, acumulatedRMSEInsertion = 0, 0, 0
    correctPredictedCharacter, totalCharactersProcessed = 0, 0
    correctPredictedFont, totalFontsProcessed = 0, 0
    
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        output = model(batch)
        
        height = output["regression_values"][:, 0]
        rotation = output["regression_values"][:, 1]
        insertion_x = output["regression_values"][:, 2]
        insertion_y = output["regression_values"][:, 3]
        
        characterLoss = F.cross_entropy(output["character_logits"], batch.y, weight=character_weights, label_smoothing=label_smoothing)
        fontLoss = F.cross_entropy(output["font_logits"], batch.font, weight=font_weights)
        heightLoss = F.mse_loss(height, batch.height.squeeze(-1))
        rotationLoss = F.mse_loss(rotation, batch.rotation.squeeze(-1))
        insertionLoss = F.mse_loss(insertion_x, batch.insertion_x.squeeze(-1)) + F.mse_loss(insertion_y, batch.insertion_y.squeeze(-1))
        
        totalLoss = characterLoss + 0.5*fontLoss + 0.5*(heightLoss + rotationLoss + insertionLoss)
        
        # backpropagation
        totalLoss.backward()
        optimizer.step()
        #scheduler.step() # CyclicLR
        
        # compute accuracy
        predictedCharacter = output["character_logits"].argmax(dim=1)
        predictedFont = output["font_logits"].argmax(dim=1)
        
        acumulatedTotalLoss += totalLoss.item()
        acumulatedCharacterLoss += characterLoss.item()
        acumulatedFontLoss += fontLoss.item()
        acumulatedHeightLoss += heightLoss.item()
        acumulatedRotationLoss += rotationLoss.item()
        acumulatedInsertionLoss += insertionLoss.item()
        
        correctPredictedCharacter += (predictedCharacter == batch.y).sum().item()
        correctPredictedFont += (predictedFont == batch.font).sum().item()
        
        totalCharactersProcessed += batch.y.size(0)
        totalFontsProcessed += batch.font.size(0)
        
        # MAE and RMSE
        predictedInsertion = torch.stack([insertion_x, insertion_y], dim=1)
        correctPredictedInsertion = torch.stack([batch.insertion_x.squeeze(-1), batch.insertion_y.squeeze(-1)], dim=1)
        acumulatedMAEHeight += F.l1_loss(height, batch.height.squeeze(-1)).item()
        acumulatedMAERotation += F.l1_loss(rotation, batch.rotation.squeeze(-1)).item()
        acumulatedMAEInsertion += F.l1_loss(predictedInsertion, correctPredictedInsertion).item()
        acumulatedRMSEHeight += torch.sqrt(F.mse_loss(height, batch.height.squeeze(-1))).item()
        acumulatedRMSERotation += torch.sqrt(F.mse_loss(rotation, batch.rotation.squeeze(-1))).item()
        acumulatedRMSEInsertion += torch.sqrt(F.mse_loss(predictedInsertion, correctPredictedInsertion)).item()
    
    # Computes epoch-wide accuracy & loss
    trainAccuracyCharacter = correctPredictedCharacter / totalCharactersProcessed
    trainAccuracyFont = correctPredictedFont / totalFontsProcessed
    
    averageTrainLossTotal = acumulatedTotalLoss / len(train_loader)
    averageTrainLossCharacter = acumulatedCharacterLoss / len(train_loader)
    averageTrainLossFont = acumulatedFontLoss / len(train_loader)
    averageTrainLossHeight = acumulatedHeightLoss / len(train_loader)
    averageTrainLossRotation = acumulatedRotationLoss / len(train_loader)
    averageTrainLossInsertion = acumulatedInsertionLoss / len(train_loader)
    
    # Evaluate clearly:
    model.eval()
    acumulatedTotalLoss = 0
    acumulatedCharacterLoss, acumulatedFontLoss = 0, 0
    acumulatedHeightLoss, acumulatedRotationLoss, acumulatedInsertionLoss = 0, 0, 0
    acumulatedMAEHeight, acumulatedMAERotation, acumulatedMAEInsertion = 0, 0, 0
    acumulatedRMSEHeight, acumulatedRMSERotation, acumulatedRMSEInsertion = 0, 0, 0
    correctPredictedCharacter, totalCharactersProcessed = 0, 0
    correctPredictedFont, totalFontsProcessed = 0, 0
    
    with torch.no_grad():
        for batch in validation_loader:
            batch = batch.to(device)
            output = model(batch)
            
            height = output["regression_values"][:, 0]
            rotation = output["regression_values"][:, 1]
            insertion_x = output["regression_values"][:, 2]
            insertion_y = output["regression_values"][:, 3]
            
            characterLoss = F.cross_entropy(output["character_logits"], batch.y, weight=character_weights, label_smoothing=label_smoothing)
            fontLoss = F.cross_entropy(output["font_logits"], batch.font, weight=font_weights)
            heightLoss = F.mse_loss(height, batch.height.squeeze(-1))
            rotationLoss = F.mse_loss(rotation, batch.rotation.squeeze(-1))
            insertionLoss = F.mse_loss(insertion_x, batch.insertion_x.squeeze(-1)) + F.mse_loss(insertion_y, batch.insertion_y.squeeze(-1))
            
            totalLoss = characterLoss + 0.5*fontLoss + 0.5*(heightLoss + rotationLoss + insertionLoss)
            
            # compute accuracy
            predictedCharacter = output["character_logits"].argmax(dim=1)
            predictedFont = output["font_logits"].argmax(dim=1)
            
            acumulatedTotalLoss += totalLoss.item()
            acumulatedCharacterLoss += characterLoss.item()
            acumulatedFontLoss += fontLoss.item()
            acumulatedHeightLoss += heightLoss.item()
            acumulatedRotationLoss += rotationLoss.item()
            acumulatedInsertionLoss += insertionLoss.item()
            
            correctPredictedCharacter += (predictedCharacter == batch.y).sum().item()
            correctPredictedFont += (predictedFont == batch.font).sum().item()
            
            totalCharactersProcessed += batch.y.size(0)
            totalFontsProcessed += batch.font.size(0)
            
            # MAE and RMSE
            predictedInsertion = torch.stack([insertion_x, insertion_y], dim=1)
            correctPredictedInsertion = torch.stack([batch.insertion_x.squeeze(-1), batch.insertion_y.squeeze(-1)], dim=1)
            acumulatedMAEHeight += F.l1_loss(height, batch.height.squeeze(-1)).item()
            acumulatedMAERotation += F.l1_loss(rotation, batch.rotation.squeeze(-1)).item()
            acumulatedMAEInsertion += F.l1_loss(predictedInsertion, correctPredictedInsertion).item()

            acumulatedRMSEHeight += torch.sqrt(F.mse_loss(height, batch.height.squeeze(-1))).item()
            acumulatedRMSERotation += torch.sqrt(F.mse_loss(rotation, batch.rotation.squeeze(-1))).item()
            acumulatedRMSEInsertion += torch.sqrt(F.mse_loss(predictedInsertion, correctPredictedInsertion)).item()        
                        
    # Computes epoch-wide accuracy & loss
    validationAccuracyCharacter = correctPredictedCharacter / totalCharactersProcessed
    validationAccuracyFont = correctPredictedFont / totalFontsProcessed
    
    averageValidationLossTotal = acumulatedTotalLoss / len(validation_loader)
    averageValidationLossCharacter = acumulatedCharacterLoss / len(validation_loader)
    averageValidationLossFont = acumulatedFontLoss / len(validation_loader)
    averageValidationLossHeight = acumulatedHeightLoss / len(validation_loader)
    averageValidationLossRotation = acumulatedRotationLoss / len(validation_loader)
    averageValidationLossInsertion = acumulatedInsertionLoss / len(validation_loader)
    
    # Adjust learning rate based on validation loss
    scheduler.step(averageValidationLossTotal) # ReduceLROnPlateau
    
    # Logging
    currentLearningRate = optimizer.param_groups[0]['lr']
    writer.add_scalar("LearningRate", currentLearningRate, epoch)
    
    writer.add_scalar("Accuracy/train_character", trainAccuracyCharacter, epoch)
    writer.add_scalar("Accuracy/train_font", trainAccuracyFont, epoch)
    
    writer.add_scalar("Accuracy/validation_character", validationAccuracyCharacter, epoch)
    writer.add_scalar("Accuracy/validation_font", validationAccuracyFont, epoch)
    
    writer.add_scalar("Loss/train_total", averageTrainLossTotal, epoch)
    writer.add_scalar("Loss/train_character", averageTrainLossCharacter, epoch)
    writer.add_scalar("Loss/train_font", averageTrainLossFont, epoch)
    writer.add_scalar("Loss/train_height", averageTrainLossHeight, epoch)
    writer.add_scalar("Loss/train_rotation", averageTrainLossRotation, epoch)
    writer.add_scalar("Loss/train_insertion_point", averageTrainLossInsertion, epoch)
    
    writer.add_scalar("Loss/validation_total", averageValidationLossTotal, epoch)
    writer.add_scalar("Loss/validation_character", averageValidationLossCharacter, epoch)
    writer.add_scalar("Loss/validation_font", averageValidationLossFont, epoch)
    writer.add_scalar("Loss/validation_height", averageValidationLossHeight, epoch)
    writer.add_scalar("Loss/validation_rotation", averageValidationLossRotation, epoch)
    writer.add_scalar("Loss/validation_insertion_point", averageValidationLossInsertion, epoch)
    
    writer.add_scalar("MAE/train_height", acumulatedMAEHeight / len(train_loader), epoch)
    writer.add_scalar("MAE/train_rotation", acumulatedMAERotation / len(train_loader), epoch)
    writer.add_scalar("MAE/train_insertion_point", acumulatedMAEInsertion / len(train_loader), epoch)
    writer.add_scalar("RMSE/train_height", acumulatedRMSEHeight / len(train_loader), epoch)
    writer.add_scalar("RMSE/train_rotation", acumulatedRMSERotation / len(train_loader), epoch)
    writer.add_scalar("RMSE/train_insertion_point", acumulatedRMSEInsertion / len(train_loader), epoch)
    
    writer.add_scalar("MAE/validation_height", acumulatedMAEHeight / len(validation_loader), epoch)
    writer.add_scalar("MAE/validation_rotation", acumulatedMAERotation / len(validation_loader), epoch)
    writer.add_scalar("MAE/validation_insertion_point", acumulatedMAEInsertion / len(validation_loader), epoch)
    writer.add_scalar("RMSE/validation_height", acumulatedRMSEHeight / len(validation_loader), epoch)
    writer.add_scalar("RMSE/validation_rotation", acumulatedRMSERotation / len(validation_loader), epoch)
    writer.add_scalar("RMSE/validation_insertion_point", acumulatedRMSEInsertion / len(validation_loader), epoch)
    
    print(f"Epoch {epoch+1} | Total Train Loss: {averageTrainLossTotal:.4f} | Train Accuracy Character: {trainAccuracyCharacter:.2f} | Total Validation Loss: {averageValidationLossTotal:.4f} | Validation Accuracy Character: {validationAccuracyCharacter:.2f}| Learning Rate: {currentLearningRate:.6f}")
    
writer.close()

print("\nTraining complete. You can now launch TensorBoard:")

Epoch 1 | Total Train Loss: 2.0501 | Train Accuracy Character: 0.84 | Total Validation Loss: 1.0239 | Validation Accuracy Character: 1.00| Learning Rate: 0.005000
Epoch 2 | Total Train Loss: 1.3707 | Train Accuracy Character: 0.96 | Total Validation Loss: 0.9867 | Validation Accuracy Character: 1.00| Learning Rate: 0.005000
Epoch 3 | Total Train Loss: 1.3324 | Train Accuracy Character: 0.96 | Total Validation Loss: 1.0264 | Validation Accuracy Character: 1.00| Learning Rate: 0.005000
Epoch 4 | Total Train Loss: 1.3720 | Train Accuracy Character: 0.96 | Total Validation Loss: 1.2261 | Validation Accuracy Character: 1.00| Learning Rate: 0.005000
Epoch 5 | Total Train Loss: 1.3486 | Train Accuracy Character: 0.97 | Total Validation Loss: 0.9601 | Validation Accuracy Character: 1.00| Learning Rate: 0.005000
Epoch 6 | Total Train Loss: 1.2914 | Train Accuracy Character: 0.97 | Total Validation Loss: 0.9545 | Validation Accuracy Character: 1.00| Learning Rate: 0.005000
Epoch 7 | Total Train 

In [ ]:
# Start tensorboard
%load_ext tensorboard
%tensorboard --logdir TensorBoard

In [6]:
# Save model to a file
save_path = "/home/rafael/Área de trabalho/Linux/OCRNetwork.pt"
torch.save(model.state_dict(), save_path)

print(f"Model saved to {save_path}")

Model saved to /home/rafael/Área de trabalho/Linux/OCRNetwork.pt
